# Analyzing FEMA's National Flood Insurance Program (NFIP) Data With DuckDB
Author: Mark Bauer

# Introduction

Refer to the [Introduction section](https://github.com/mebauer/duckdb-fema-nfip) on the project's main page.

# OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2. Retrieved from https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA's  [Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

**Dataset Description**:
>This dataset provides details on NFIP claims transactions. It is derived from the NFIP system of record, staged in the NFIP reporting platform and redacted to protect policy holder personally identifiable information.
>
>This dataset is not intended to be an official federal report, and should not be considered an official federal report.

**About the National Flood Insurance Program**:   
>Congress passed the National Flood Insurance Act (NFIA), 42 U.S.C. 4001 in 1968, creating the National Flood Insurance Program (NFIP) in order to reduce future flood losses through flood hazard identification, floodplain management, and providing insurance protection. The Department of Housing and Urban Development (HUD) originally administered the NFIP, and Congress subsequently transferred the NFIP to FEMA upon its creation in 1979. FEMA and insurance companies participating in FEMA's Write Your Own (WYO) program offer NFIP insurance coverage for building structures as well as for contents and personal property within the building structures to eligible and insurable properties. The WYO program began in 1983 with NFIP operating under Part B of the NFIA and allows FEMA to authorize private insurance companies to issue the Standard Flood Insurance Policy (SFIP) as FEMA's fiduciary and fiscal agent. FEMA administers NFIP by ensuring insurance applications are processed properly; determining correct premiums; renewing, reforming, and cancelling insurance policies; transferring policies from the seller of the property to the purchaser of the property in certain circumstances; and processing insurance claims.
>
>The paid premiums of SFIPs and claims payments for damaged property are processed through the National Flood Insurance Fund (NFIF). NFIF was established by the National Flood Insurance Act of 1968 (42 U.S.C. 4001, et seq.), and is a centralized premium revenue and fee-generated fund that supports NFIP, which holds these U.S. Treasury funds.
>
>The Flood Insurance Claims Manual (https://nfipservices.floodsmart.gov/insurance-manuals) provides claims guidance to WYOs, vendors, adjusters, and examiners so that policyholders experience consistent and reliable service. The Manual provides processes for handling claims from the notice of loss to final payment. The NFIP has provided answers to Frequently Asked Questions (FAQs) to assist the public in understanding and navigating the data our program makes available: https://www.fema.gov/sites/default/files/documents/fema_nfip-data-faqs.pdf.

**Data Dictionary**:  
View the data dictionary on OpenFEMA under the [Data Fields](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) section.

In [1]:
# import libraries
import duckdb
import pandas as pd

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb,pandas

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0
pandas: 1.5.1



In [3]:
# examine datasets
%ls data/

FimaNfipClaims.csv


# Redacted Claims: Before Getting Started
Please note that this dataset provides details on NFIP claims transactions and is ***redacted*** to protect policy holder personally identifiable information. The claim's `latitude` and `longitude` field **should not be used to represent the precise location of the insured building**. From the data dictionary:

>Latitude: **Approximate latitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.
>
>Longitude: **Approximate longitude of the insured building (to 1 decimal place)**. This represents the approximate location of the insured property. The precision has been lessened to ensure individual privacy. This may result in a point location that exists in an incorrect county or state. Use the state and county fields for record aggregation for these dimensions.

For more information, visit the [Data Dictionary](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2) and review [Frequently Asked Questions about NFIP Policies and Claims Data](https://nfipservices.floodsmart.gov/frequently-asked-questions-about-nfip-policies-and-claims-data).

This analysis presents all financial figures in nominal dollars. No adjustments for inflation have been made, meaning that the values reflect the actual dollar amounts at the time of measurement. Additionally, the date and time of data access can be found in the download-data notebook.

# Create a DuckDB database instance using the Python client

In [4]:
# create a DuckDB database instance
con = duckdb.connect()

# create table claims of dataset
con.execute("""
    CREATE TABLE claims AS
        FROM read_csv('data/FimaNfipClaims.csv')
""")

# sanity check
con.sql("""
    SELECT *
    FROM claims
    LIMIT 5
""").show()

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────┬───────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ basementEnclosureC…  │ … │ latitude │ longitude │          id          │
│        int64         │      timestamp       │        int64         │   │  double  │  double   │       varchar        │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────┼───────────┼──────────────────────┤
│                    0 │ 2020-11-13 14:50:3…  │                    2 │ … │     30.3 │     -87.7 │ e7af3d9f-b605-4653…  │
│                    0 │ 2020-12-11 16:25:4…  │                    0 │ … │     30.3 │     -87.7 │ bbaeaf64-c162-41bf…  │
│                    0 │ 2020-03-27 12:15:4…  │                    0 │ … │     33.5 │    -112.1 │ 256da746-b30b-4129…  │
│                    0 │ 2020-03-27 12:15:4…  │                    0 │ … │     33.5 │    -112.1 │ e3dcbb27-a2d0-4a9e…  │
│                    0 │ 2020-03

In [5]:
# list tables and schemas
con.sql("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,memory,main,claims,"[agricultureStructureIndicator, asOfDate, base...","[BIGINT, TIMESTAMP, BIGINT, BIGINT, BIGINT, TI...",False


In [6]:
# count of rows
con.sql("""
    SELECT
        COUNT(*) AS count_rows
    FROM
        claims
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│    2671486 │
└────────────┘

In [7]:
# count of columns
con.sql("""
    SELECT
        COUNT(column_name) AS count_columns
    FROM
        (DESCRIBE FROM claims)
""")

┌───────────────┐
│ count_columns │
│     int64     │
├───────────────┤
│            73 │
└───────────────┘

In [8]:
# last refreshed
con.sql("""
    SELECT
        asOfDate
    FROM
        claims
    ORDER BY
        asOfDate DESC
    LIMIT 1
""")

┌─────────────────────────┐
│        asOfDate         │
│        timestamp        │
├─────────────────────────┤
│ 2024-10-04 00:59:55.552 │
└─────────────────────────┘

# Examine Dataset

## Column Info

In [9]:
# examine column datatypes
con.sql("""
    SELECT
        column_name,
        column_type
    FROM
        (DESCRIBE claims)
""").show(max_rows=80)

┌────────────────────────────────────────────┬─────────────┐
│                column_name                 │ column_type │
│                  varchar                   │   varchar   │
├────────────────────────────────────────────┼─────────────┤
│ agricultureStructureIndicator              │ BIGINT      │
│ asOfDate                                   │ TIMESTAMP   │
│ basementEnclosureCrawlspaceType            │ BIGINT      │
│ policyCount                                │ BIGINT      │
│ crsClassificationCode                      │ BIGINT      │
│ dateOfLoss                                 │ TIMESTAMP   │
│ elevatedBuildingIndicator                  │ BIGINT      │
│ elevationCertificateIndicator              │ VARCHAR     │
│ elevationDifference                        │ DOUBLE      │
│ baseFloodElevation                         │ DOUBLE      │
│ ratedFloodZone                             │ VARCHAR     │
│ houseWorship                               │ BIGINT      │
│ locationOfContents    

In [10]:
# approximate column null percentage
con.sql("""
    SELECT
        column_name,
        null_percentage
    FROM
        (SUMMARIZE FROM claims)
    WHERE
        null_percentage > 0
    ORDER BY
        null_percentage DESC
""").show(max_rows=80)

┌────────────────────────────────────────────┬─────────────────┐
│                column_name                 │ null_percentage │
│                  varchar                   │  decimal(9,2)   │
├────────────────────────────────────────────┼─────────────────┤
│ floodCharacteristicsIndicator              │           98.51 │
│ eventDesignationNumber                     │           94.34 │
│ lowestAdjacentGrade                        │           81.26 │
│ crsClassificationCode                      │           80.65 │
│ nonPaymentReasonBuilding                   │           78.02 │
│ elevationCertificateIndicator              │           77.72 │
│ lowestFloorElevation                       │           76.38 │
│ baseFloodElevation                         │           75.70 │
│ elevationDifference                        │           72.95 │
│ floodZoneCurrent                           │           72.15 │
│ nfipCommunityNumberCurrent                 │           72.10 │
│ nfipCommunityName      

## Preview Data

In [11]:
# preview data
con.sql("""
    SELECT *
    FROM claims
    LIMIT 5
""").show()

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────┬───────────┬──────────────────────┐
│ agricultureStructu…  │       asOfDate       │ basementEnclosureC…  │ … │ latitude │ longitude │          id          │
│        int64         │      timestamp       │        int64         │   │  double  │  double   │       varchar        │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────┼───────────┼──────────────────────┤
│                    0 │ 2020-11-13 14:50:3…  │                    2 │ … │     30.3 │     -87.7 │ e7af3d9f-b605-4653…  │
│                    0 │ 2020-12-11 16:25:4…  │                    0 │ … │     30.3 │     -87.7 │ bbaeaf64-c162-41bf…  │
│                    0 │ 2020-03-27 12:15:4…  │                    0 │ … │     33.5 │    -112.1 │ 256da746-b30b-4129…  │
│                    0 │ 2020-03-27 12:15:4…  │                    0 │ … │     33.5 │    -112.1 │ e3dcbb27-a2d0-4a9e…  │
│                    0 │ 2020-03

In [12]:
# preview data as pandas dataframe for readability
sql = """
    SELECT *
    FROM claims
    LIMIT 5
"""

# examine each column in sections because of large number of columns
con.sql(sql).df().iloc[:, :15]

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation
0,0,2020-11-13 14:50:38.288,2,1,NaN,2020-09-16,0,None,6.0,10.0,AE,0,NaN,0.0,16.0
1,0,2020-12-11 16:25:40.587,0,1,NaN,2020-09-16,1,None,4.0,10.0,AE,0,7.0,4.4,14.4
2,0,2020-03-27 12:15:45.887,0,1,NaN,2016-08-02,0,2,NaN,NaN,AH,0,3.0,NaN,NaN
3,0,2020-03-27 12:15:45.887,0,1,NaN,2014-09-08,0,2,NaN,NaN,AH,0,3.0,NaN,NaN
4,0,2020-03-26 12:56:27.476,0,1,NaN,2018-01-09,1,None,NaN,NaN,AE,0,NaN,NaN,NaN


In [13]:
# slice through columns
con.sql(sql).df().iloc[:, 15:30]

,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss
0,2,0,NaN,2,1975-01-01,2008-10-20,2695.44,0.0,0.0,1,8,0,7400,0,2020
1,3,0,54.0,1,1991-09-17,2017-10-05,2750.18,121.0,0.0,1,8,0,10800,700,2020
2,1,0,NaN,1,1949-01-01,2014-09-02,NaN,NaN,NaN,0,B,0,153500,15800,2016
3,1,0,NaN,1,1949-01-01,2014-09-02,25156.27,0.0,0.0,0,B,0,139500,15000,2014
4,1,0,10.0,1,1960-01-01,2001-08-05,NaN,NaN,NaN,0,1,0,250000,0,2018


In [14]:
# slice through columns
con.sql(sql).df().iloc[:, 30:45]

,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator
0,0,3945.0,F,2695.44,167531.0,1,N,0.0,None,0.0,0.0,0,AL0520,NaN,NaN
1,1,4000.0,F,2750.18,278027.0,1,N,1371.0,F,121.0,50000.0,0,AL0520,NaN,NaN
2,1,1698.0,5,0.00,140422.0,4,N,3690.0,5,0.0,21500.0,0,None,NaN,NaN
3,1,30098.0,5,25156.27,140919.0,4,N,NaN,5,0.0,NaN,0,None,NaN,NaN
4,1,NaN,2,0.00,NaN,4,N,NaN,0,0.0,NaN,0,None,368.0,NaN


In [15]:
# slice through columns
con.sql(sql).df().iloc[:, 45:60]

,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator
0,NaN,0,Hurricane Sally,30000,0.0,015005,0000,None,None,None,0,185457.0,0.0,A,0
1,NaN,0,Hurricane Sally,30000,0.0,015005,0000,None,None,None,0,314835.0,75000.0,A,0
2,0.0,0,None,30000,0.0,040051,0000,None,01,01,0,175528.0,0.0,R,0
3,0.0,0,None,30000,0.0,040051,0000,None,None,None,0,173974.0,NaN,R,0
4,0.0,0,Mid-Winter California Flooding,30000,0.0,060331,0000,None,None,06,0,NaN,NaN,A,0


In [16]:
# slice through columns
con.sql(sql).df().iloc[:, 60:]

,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,None,1,0,AL,Currently Unavailable,36542,01003,01003011407,010030114073,30.3,-87.7,e7af3d9f-b605-4653-958f-bc48f413766c
1,0,AE,1,0,AL,Currently Unavailable,36542,01003,01003011407,010030114072,30.3,-87.7,bbaeaf64-c162-41bf-a399-862d0e832447
2,1,None,1,0,AZ,Currently Unavailable,85015,04013,04013108902,040131089022,33.5,-112.1,256da746-b30b-4129-a391-93f50a6190c8
3,1,None,1,0,AZ,Currently Unavailable,85015,04013,04013108902,040131089022,33.5,-112.1,e3dcbb27-a2d0-4a9e-a70d-ff7e578d4e6a
4,0,AE,1,0,CA,Currently Unavailable,93108,06083,06083001402,060830014022,34.4,-119.6,f77efb94-0188-4fa0-b52e-3aa71499baeb


In [17]:
# count duplicate IDs
con.sql("""
    SELECT
        id,
        COUNT(id) AS count
    FROM
        claims
    GROUP BY
        id
    HAVING
        count > 1
""").show()

┌─────────┬───────┐
│   id    │ count │
│ varchar │ int64 │
├─────────┴───────┤
│     0 rows      │
└─────────────────┘



In [18]:
# earliest and latest record effective date
con.sql("""
    SELECT
        min(asOfDate) AS earliestAsOfDate,
        max(asOfDate) AS latestAsOfDate
    FROM
        claims
""").show()

┌─────────────────────────┬─────────────────────────┐
│    earliestAsOfDate     │     latestAsOfDate      │
│        timestamp        │        timestamp        │
├─────────────────────────┼─────────────────────────┤
│ 2019-09-19 06:12:43.388 │ 2024-10-04 00:59:55.552 │
└─────────────────────────┴─────────────────────────┘



In [19]:
# top 5 most recent effective date claim records
con.sql("""
    SELECT
        asOfDate,
        dateOfLoss,
        floodEvent,
        state,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim)::BIGINT AS paidTotalClaim
    FROM
        claims
    ORDER BY
        asOfDate DESC
    LIMIT 5
""").show()

┌─────────────────────────┬─────────────────────┬────────────────────────┬─────────┬────────────────┐
│        asOfDate         │     dateOfLoss      │       floodEvent       │  state  │ paidTotalClaim │
│        timestamp        │      timestamp      │        varchar         │ varchar │     int64      │
├─────────────────────────┼─────────────────────┼────────────────────────┼─────────┼────────────────┤
│ 2024-10-04 00:59:55.552 │ 2024-09-16 00:00:00 │ Tropical Cyclone Eight │ NC      │              0 │
│ 2024-10-04 00:58:55.563 │ 2024-09-11 00:00:00 │ Hurricane Francine     │ LA      │           2901 │
│ 2024-10-04 00:57:54.885 │ 2024-09-11 00:00:00 │ Hurricane Francine     │ LA      │              0 │
│ 2024-10-04 00:57:54.695 │ 2024-09-26 00:00:00 │ Hurricane Helene       │ FL      │           7500 │
│ 2024-10-04 00:56:54.47  │ 2024-09-11 00:00:00 │ Hurricane Francine     │ LA      │          57179 │
└─────────────────────────┴─────────────────────┴────────────────────────┴────────

In [20]:
# earliest and latest date of loss in dataset
con.sql("""
    SELECT
        min(dateOfLoss) AS earliestDateOfLoss,
        max(dateOfLoss) AS latestDateOfLoss
    FROM
        claims
""").show()

┌─────────────────────┬─────────────────────┐
│ earliestDateOfLoss  │  latestDateOfLoss   │
│      timestamp      │      timestamp      │
├─────────────────────┼─────────────────────┤
│ 1978-01-01 00:00:00 │ 2024-10-03 00:00:00 │
└─────────────────────┴─────────────────────┘



In [21]:
# top 5 latest claim records by date of loss 
con.sql("""
    SELECT
        dateOfLoss,
        asOfDate,
        floodEvent,
        state,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim)::BIGINT AS paidTotalClaim
    FROM
        claims
    ORDER BY
        dateOfLoss DESC, asOfDate DESC
    LIMIT 5
""").show()

┌─────────────────────┬─────────────────────────┬────────────┬─────────┬────────────────┐
│     dateOfLoss      │        asOfDate         │ floodEvent │  state  │ paidTotalClaim │
│      timestamp      │        timestamp        │  varchar   │ varchar │     int64      │
├─────────────────────┼─────────────────────────┼────────────┼─────────┼────────────────┤
│ 2024-10-03 00:00:00 │ 2024-10-03 22:38:35.955 │ NULL       │ FL      │           NULL │
│ 2024-10-03 00:00:00 │ 2024-10-03 21:40:08.5   │ NULL       │ FL      │           NULL │
│ 2024-10-03 00:00:00 │ 2024-10-03 17:21:21.871 │ NULL       │ FL      │           NULL │
│ 2024-10-02 00:00:00 │ 2024-10-03 21:40:08.5   │ NULL       │ FL      │           NULL │
│ 2024-10-02 00:00:00 │ 2024-10-03 21:40:08.5   │ NULL       │ FL      │           NULL │
└─────────────────────┴─────────────────────────┴────────────┴─────────┴────────────────┘



In [22]:
# total insured units in dataset
con.sql("""
    SELECT
        SUM(policyCount) AS totalPolicyCount
    FROM
        claims
""").show()

┌──────────────────┐
│ totalPolicyCount │
│      int128      │
├──────────────────┤
│          3462669 │
└──────────────────┘



Policy Count:
>Insured units in an active status. A policy contract ceases to be in an active status as of the cancellation date or the expiration date. Residential Condominium Building Association Policy (RCBAP) contracts are stored as a single policy contract but insure multiple units and therefore represent multiple policies.

## Summary Statistics

In [23]:
# calculate summary statistics of each column, pandas df for readability
con.sql("""
    SELECT *
    FROM (SUMMARIZE claims)
""").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,agricultureStructureIndicator,BIGINT,0,1,2,0.00014598616650059182,0.012081593817782434,0,0,0,2671486,0.00
1,asOfDate,TIMESTAMP,2019-09-19 06:12:43.388,2024-10-04 00:59:55.552,479449,None,None,None,None,None,2671486,0.00
2,basementEnclosureCrawlspaceType,BIGINT,0,4,4,1.195105377264796,1.0618378842847185,0,1,2,2671486,69.92
3,policyCount,BIGINT,1,1090,399,1.2961583927447122,6.742744486702049,1,1,1,2671486,0.00
4,crsClassificationCode,BIGINT,1,10,10,6.708258215272243,1.526304305976192,5,7,8,2671486,80.65
...,...,...,...,...,...,...,...,...,...,...,...,...
68,censusTract,VARCHAR,01001020100,78030961200,59577,None,None,None,None,None,2671486,5.16
69,censusBlockGroupFips,VARCHAR,010010201001,780309612002,117722,None,None,None,None,None,2671486,5.16
70,latitude,DOUBLE,-36.0,69.9,334,33.755900244380626,5.827326735346257,29.807589746046872,30.645513794090203,39.66243830088295,2671486,1.51
71,longitude,DOUBLE,-170.7,174.0,715,-86.02134344014759,10.940268931584674,-91.13371113348664,-86.49085404693089,-77.44163542302745,2671486,1.51


In [24]:
# build summarize relation
summarize_relation = con.sql("SELECT * FROM (SUMMARIZE claims)")

# preview relation
summarize_relation.show()

┌──────────────────────┬─────────────┬──────────────────────┬───┬────────────────────┬─────────┬─────────────────┐
│     column_name      │ column_type │         min          │ … │        q75         │  count  │ null_percentage │
│       varchar        │   varchar   │       varchar        │   │      varchar       │  int64  │  decimal(9,2)   │
├──────────────────────┼─────────────┼──────────────────────┼───┼────────────────────┼─────────┼─────────────────┤
│ agricultureStructu…  │ BIGINT      │ 0                    │ … │ 0                  │ 2671486 │            0.00 │
│ asOfDate             │ TIMESTAMP   │ 2019-09-19 06:12:4…  │ … │ NULL               │ 2671486 │            0.00 │
│ basementEnclosureC…  │ BIGINT      │ 0                    │ … │ 2                  │ 2671486 │           69.92 │
│ policyCount          │ BIGINT      │ 1                    │ … │ 1                  │ 2671486 │            0.00 │
│ crsClassificationC…  │ BIGINT      │ 1                    │ … │ 8             

In [25]:
# slice through columns for readability
con.sql("SELECT * FROM summarize_relation").df().iloc[:15, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,agricultureStructureIndicator,BIGINT,0,1,2,0.00014598616650059182,0.012081593817782368,0,0,0,2671486,0.00
1,asOfDate,TIMESTAMP,2019-09-19 06:12:43.388,2024-10-04 00:59:55.552,479449,None,None,None,None,None,2671486,0.00
2,basementEnclosureCrawlspaceType,BIGINT,0,4,4,1.195105377264796,1.0618378842847207,0,1,2,2671486,69.92
3,policyCount,BIGINT,1,1090,399,1.2961583927447122,6.74274448670205,1,1,1,2671486,0.00
4,crsClassificationCode,BIGINT,1,10,10,6.708258215272243,1.526304305976192,5,7,8,2671486,80.65
5,dateOfLoss,TIMESTAMP,1978-01-01 00:00:00,2024-10-03 00:00:00,16868,None,None,None,None,None,2671486,0.00
6,elevatedBuildingIndicator,BIGINT,0,1,2,0.18133877549798127,0.38529868877158163,0,0,0,2671486,0.00
7,elevationCertificateIndicator,VARCHAR,1,E,9,None,None,None,None,None,2671486,77.72
8,elevationDifference,DOUBLE,-9989.0,998.0,371,1.253409617633019,29.091407737413885,0.0,1.0,2.7474439219632734,2671486,72.95
9,baseFloodElevation,DOUBLE,-9999.0,9998.0,10427,126.73271926107897,767.4360045962655,6.808632423063275,9.00039181829393,13.93652892150554,2671486,75.70


In [26]:
# slice through columns for readability
con.sql("SELECT * FROM summarize_relation").df().iloc[15:30, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
15,numberOfFloorsInTheInsuredBuilding,BIGINT,1,6,6,1.6958577188631916,0.8748596361444527,1,1,2,2671486,0.65
16,nonProfitIndicator,BIGINT,0,1,2,0.0013876172287633176,0.0372248877216163,0,0,0,2671486,0.00
17,obstructionType,BIGINT,10,98,20,16.553431553900655,16.107515135628212,10,10,10,2671486,43.74
18,occupancyType,BIGINT,1,19,14,1.9216470466664222,2.5068800508373754,1,1,1,2671486,0.02
19,originalConstructionDate,TIMESTAMP,1201-01-01 00:00:00,2024-09-16 00:00:00,31912,None,None,None,None,None,2671486,0.03
20,originalNBDate,TIMESTAMP,0998-09-21 00:00:00,2024-11-09 00:00:00,17742,None,None,None,None,None,2671486,0.00
21,amountPaidOnBuildingClaim,DOUBLE,-162432.16,10006145.71,1286108,31531.109649180973,66190.17745513326,2317.1640557646992,9633.387051613568,36534.989849357065,2671486,22.78
22,amountPaidOnContentsClaim,DOUBLE,-41276.32,757048.95,472789,7096.517735698417,22183.209325532247,0.0,1.7398585225357502e-06,4796.289056966857,2671486,22.78
23,amountPaidOnIncreasedCostOfComplianceClaim,DOUBLE,-6450.0,60000.0,8752,459.7303803291179,3489.7836171167105,0.0,0.0,0.0,2671486,22.78
24,postFIRMConstructionIndicator,BIGINT,0,1,2,0.26588759963555864,0.441804772564026,0,0,1,2671486,0.00


In [27]:
# slice through columns for readability
con.sql("SELECT * FROM summarize_relation").df().iloc[30:45, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
30,primaryResidenceIndicator,BIGINT,0,1,2,0.4892947221134604,0.4998854774479687,0,0,1,2671486,0.00
31,buildingDamageAmount,BIGINT,0,927700000,202772,36354.07504655704,812334.7780210077,3429,11228,40125,2671486,23.76
32,buildingDeductibleCode,VARCHAR,0,H,15,None,None,None,None,None,2671486,11.77
33,netBuildingPaymentAmount,DOUBLE,-162432.16,10006145.71,1286178,24319.109697074964,59591.4128871143,0.004801795055906804,4445.082100124046,23992.325530881346,2671486,0.00
34,buildingPropertyValue,BIGINT,0,9950000000,420035,7148318.887278207,199917804.10704142,60768,109891,189858,2671486,23.76
35,causeOfDamage,VARCHAR,0,Z,17,None,None,None,None,None,2671486,2.26
36,condominiumCoverageTypeCode,VARCHAR,A,U,5,None,None,None,None,None,2671486,1.40
37,contentsDamageAmount,BIGINT,0,19230507,102299,18331.886239398373,86944.02029162779,1611,5682,18353,2671486,59.72
38,contentsDeductibleCode,VARCHAR,0,H,15,None,None,None,None,None,2671486,20.56
39,netContentsPaymentAmount,DOUBLE,-41276.32,757048.95,473195,5458.730475039014,19697.04273977996,0.0,0.0,2115.4654194716413,2671486,0.00


In [28]:
# slice through columns for readability
con.sql("SELECT * FROM summarize_relation").df().iloc[45:60, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
45,floodWaterDuration,BIGINT,0,999,261,0.8068743267334517,14.72863591340925,0,0,0,2671486,9.06
46,floodproofedIndicator,BIGINT,0,1,2,0.0001411199609505721,0.011878556264305687,0,0,0,2671486,0.00
47,floodEvent,VARCHAR,2021 Mid-Spring Severe Storms,Yellowstone Flooding,177,None,None,None,None,None,2671486,28.48
48,iccCoverage,BIGINT,0,30000,4,27991.868422816147,4754.676432391682,30000,30000,30000,2671486,36.32
49,netIccPaymentAmount,DOUBLE,-6450.0,60000.0,8709,353.6949859478955,3066.941960282361,0.0,0.0,0.0,2671486,0.00
50,nfipRatedCommunityNumber,VARCHAR,000000,999999,16299,None,None,None,None,None,2671486,0.00
51,nfipCommunityNumberCurrent,VARCHAR,0000,815000,12065,None,None,None,None,None,2671486,72.10
52,nfipCommunityName,VARCHAR,ABBEVILLE COUNTY *,"ZUMBRO FALLS, CITY OF",9870,None,None,None,None,None,2671486,70.76
53,nonPaymentReasonContents,VARCHAR,01,99,23,None,None,None,None,None,2671486,68.95
54,nonPaymentReasonBuilding,VARCHAR,01,99,23,None,None,None,None,None,2671486,78.02


In [29]:
# slice through columns for readability
con.sql("SELECT * FROM summarize_relation").df().iloc[60:, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
60,waterDepth,BIGINT,-999,999,478,4.289864898924166,16.454985259103754,0,1,2,2671486,9.40
61,floodZoneCurrent,VARCHAR,A,X,62,None,None,None,None,None,2671486,72.15
62,buildingDescriptionCode,BIGINT,1,21,18,1.3360927447287696,1.4946867436844309,1,1,1,2671486,64.28
63,rentalPropertyIndicator,BIGINT,0,1,2,0.012453742973012024,0.11089928702497898,0,0,0,2671486,0.00
64,state,VARCHAR,AK,WY,57,None,None,None,None,None,2671486,0.00
65,reportedCity,VARCHAR,Currently Unavailable,Currently Unavailable,1,None,None,None,None,None,2671486,0.00
66,reportedZipCode,VARCHAR,00000,99999,26326,None,None,None,None,None,2671486,2.09
67,countyCode,VARCHAR,01001,78030,2946,None,None,None,None,None,2671486,2.32
68,censusTract,VARCHAR,01001020100,78030961200,59577,None,None,None,None,None,2671486,5.16
69,censusBlockGroupFips,VARCHAR,010010201001,780309612002,117722,None,None,None,None,None,2671486,5.16


## Highlighted Features: Amount Paid on Claims Summary Statistics
We will analyze these features extensively throughout this notebook.

**amountPaidOnBuildingClaim**: Dollar amount paid on the building claim. In some instances, a negative amount may appear which occurs when a check issued to a policy holder is not cashed and has to be re-issued.

**amountPaidOnContentsClaim**: Dollar amount paid on the contents claim. In some instances, a negative amount may appear, which occurs when a check issued to a policy holder is not cashed and has to be re-issued.

**amountPaidOnIncreasedCostOfComplianceClaim**: ICC coverage is one of several flood insurance resources for policyholders who need additional help rebuilding after a flood. It provides up to $30,000 to help cover the cost of mitigation measures that will reduce the flood risk.

Source: [Data Dictionary](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2)

In [30]:
# examine summary statistics on paid total claims
con.sql("""
    SELECT
        column_name, column_type, count, null_percentage,
        min, max, approx_unique,
        ROUND(avg::DOUBLE, 2) AS avg,
        ROUND(std::DOUBLE, 2) AS std,
        ROUND(q25::DOUBLE, 2) AS q25,
        ROUND(q50::DOUBLE, 2) AS q50,
        ROUND(q75::DOUBLE, 2) AS q75
    FROM
        summarize_relation
    WHERE column_name IN (
        'amountPaidOnBuildingClaim',
        'amountPaidOnContentsClaim',
        'amountPaidOnIncreasedCostOfComplianceClaim'
    )
""").df()

,column_name,column_type,count,null_percentage,min,max,approx_unique,avg,std,q25,q50,q75
0,amountPaidOnBuildingClaim,DOUBLE,2671486,22.78,-162432.16,10006145.71,1286108,31531.11,66190.18,2320.28,9618.31,36673.21
1,amountPaidOnContentsClaim,DOUBLE,2671486,22.78,-41276.32,757048.95,472789,7096.52,22183.21,0.00,0.00,4771.77
2,amountPaidOnIncreasedCostOfComplianceClaim,DOUBLE,2671486,22.78,-6450.0,60000.0,8752,459.73,3489.78,0.00,0.00,0.00


In [31]:
# top 10 paid total claim records
con.sql("""
    SELECT
        dateOfLoss,
        floodEvent,
        state,
        policyCount,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS paidTotalClaim,
        ROUND(amountPaidOnBuildingClaim, 0)::INT AS amountPaidOnBuildingClaim,
        ROUND(amountPaidOnContentsClaim, 0)::INT AS amountPaidOnContentsClaim,
        ROUND(amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS amountPaidICC
    FROM
        claims
    ORDER BY
        paidTotalClaim DESC
    LIMIT 10
""").df()

,dateOfLoss,floodEvent,state,policyCount,paidTotalClaim,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidICC
0,2022-09-28,Hurricane Ian,FL,198,10106146,10006146,100000,0
1,2005-08-29,Hurricane Katrina,MS,48,10000000,10000000,0,0
2,2012-10-29,Hurricane Sandy,NY,184,9467720,9467720,0,0
3,2004-09-15,Hurricane Ivan,FL,64,9169507,9100033,39474,30000
4,2001-06-09,Tropical Storm Allison,TX,233,9023558,8973270,50288,0
5,2022-09-28,Hurricane Ian,FL,36,9000000,9000000,0,0
6,2022-09-28,Hurricane Ian,FL,54,8734196,8734196,0,0
7,2005-08-29,Hurricane Katrina,MS,71,8386100,8349000,37100,0
8,2022-09-28,Hurricane Ian,FL,51,7778937,7778937,0,0
9,2012-10-29,Hurricane Sandy,NY,76,7447962,7447962,0,0


In [32]:
# top 10 paid total claim records for only one policy
con.sql("""
    SELECT
        dateOfLoss,
        floodEvent,
        state,
        policyCount,
        ROUND(
            amountPaidOnBuildingClaim
            + amountPaidOnContentsClaim
            + amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS paidTotalClaim,
        ROUND(amountPaidOnBuildingClaim, 0)::INT AS amountPaidOnBuildingClaim,
        ROUND(amountPaidOnContentsClaim, 0)::INT AS amountPaidOnContentsClaim,
        ROUND(amountPaidOnIncreasedCostOfComplianceClaim, 0)::INT AS amountPaidICC
    FROM
        claims
    WHERE
        policyCount == 1
    ORDER BY
        paidTotalClaim DESC
    LIMIT 10
""").df()

,dateOfLoss,floodEvent,state,policyCount,paidTotalClaim,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidICC
0,2008-06-13,None,IA,1,1250000,500000,750000,0
1,2023-04-13,April Florida Flooding,FL,1,1065210,1000000,65210,0
2,2016-05-24,None,AR,1,1054205,500000,554205,0
3,2012-10-29,Hurricane Sandy,NJ,1,1050000,500000,550000,0
4,2012-10-29,Hurricane Sandy,NJ,1,1039512,500000,539512,0
5,1998-09-27,Hurricane Georges,AL,1,1000000,500000,500000,0
6,2004-09-16,Hurricane Ivan,AL,1,1000000,500000,500000,0
7,2004-09-16,Hurricane Ivan,AL,1,1000000,500000,500000,0
8,2003-09-16,None,DE,1,1000000,500000,500000,0
9,2003-09-15,None,DE,1,1000000,500000,500000,0


## Highlighted Features: Building Damage, Values, and Replacement Costs Summary Statistics

In [33]:
# examine summary statistics on building damage amounts, property values and replacement costs columns
con.sql("""
    SELECT
        column_name, column_type, count, null_percentage,
        min, max, approx_unique,
        ROUND(avg::DOUBLE, 2) AS avg,
        ROUND(std::DOUBLE, 2) AS std,
        ROUND(q25::DOUBLE, 2) AS q25,
        ROUND(q50::DOUBLE, 2) AS q50,
        ROUND(q75::DOUBLE, 2) AS q75,
    FROM
        summarize_relation
    WHERE column_name IN (
        'buildingPropertyValue',
        'buildingReplacementCost',
        'buildingDamageAmount'
    )
""").df()

,column_name,column_type,count,null_percentage,min,max,approx_unique,avg,std,q25,q50,q75
0,buildingPropertyValue,BIGINT,2671486,23.76,0,9950000000,420035,7148318.89,1.999178e+08,60627.0,109791.0,190002.0
1,buildingReplacementCost,BIGINT,2671486,23.76,0,9999000000,447053,8569836.74,2.350055e+08,0.0,120648.0,224076.0
2,buildingDamageAmount,BIGINT,2671486,23.76,0,927700000,202772,36354.08,8.123348e+05,3439.0,11218.0,40195.0


## Highlighted Features: Elevation and Water Depth Summary Statistics

In [34]:
# examine summary statisitics on elevation and water depth columns
con.sql("""
    SELECT
        column_name, column_type, count, null_percentage,
        min, max, approx_unique,
        ROUND(avg::DOUBLE, 2) AS avg,
        ROUND(std::DOUBLE, 2) AS std,
        ROUND(q25::DOUBLE, 2) AS q25,
        ROUND(q50::DOUBLE, 2) AS q50,
        ROUND(q75::DOUBLE, 2) AS q75
    FROM
        summarize_relation
    WHERE column_name IN (
        'baseFloodElevation',
        'waterDepth',
        'lowestAdjacentGrade',
        'lowestFloorElevation',
        'elevationDifference',
    )
""").df()

,column_name,column_type,count,null_percentage,min,max,approx_unique,avg,std,q25,q50,q75
0,elevationDifference,DOUBLE,2671486,72.95,-9989.0,998.0,371,1.25,29.09,0.00,1.00,2.78
1,baseFloodElevation,DOUBLE,2671486,75.70,-9999.0,9998.0,10427,126.73,767.44,6.85,9.00,13.88
2,lowestAdjacentGrade,DOUBLE,2671486,81.26,-99999.9,9998.9,12638,49.70,1483.25,3.15,6.08,10.77
3,lowestFloorElevation,DOUBLE,2671486,76.38,-9999.0,9998.9,13294,104.33,616.41,6.55,10.27,17.33
4,waterDepth,BIGINT,2671486,9.40,-999,999,478,4.29,16.45,0.00,1.00,2.00


# Analysis

## Total Paid Claims

In [35]:
con.sql("""
    SELECT
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
""").df()

,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,2671486,80638352603,65049593608,14640321888,948437107


## Top 20 States by Total Paid Claims

In [36]:
con.sql("""
    SELECT
        state,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim, 
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        state
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,state,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,LA,483956,20869805385,16459981751,4132267467,277556167
1,TX,390889,17218491455,13253973375,3906593054,57925025
2,FL,415410,11433783265,9699132485,1696552579,38098201
3,NJ,201234,6470109817,5349181327,870522071,250406420
4,NY,174823,5727350837,4955149755,719127080,53074002
5,MS,64129,3122839913,2386323813,675594228,60921873
6,NC,107850,2009961692,1698489347,274446907,37025438
7,PA,76664,1432141143,1123943137,293019403,15178602
8,AL,44754,1192064063,966786458,212390245,12887360
9,SC,49288,1017326931,882236727,131285402,3804802


## Top 20 Counties by Total Paid Claims

In [37]:
con.sql("""
    SELECT
        state,
        countyCode,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        state, countyCode
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,state,countyCode,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,TX,48201,170625,8748468467,6794800334,1933648655,20019478
1,LA,22071,127137,7296273827,5892518413,1293261898,110493515
2,LA,22051,135361,3542723793,2633465878,847684749,61573166
3,FL,12071,43354,3413583297,3050925028,360290319,2367950
4,NJ,34029,52795,2607964807,2160867823,275256283,171840701
5,TX,48167,60552,2466669297,1896169966,549528562,20970769
6,NY,36059,51480,2279955547,1972897371,290304539,16753636
7,LA,22087,23913,2245090178,1680098976,553956976,11034225
8,LA,22103,38287,1775723597,1378279810,383168918,14274869
9,LA,22033,22493,1311706040,1140369431,169389761,1946847


## Top 20 Flood Events by Total Paid Claims

In [38]:
con.sql("""
    SELECT
        floodEvent,
        yearOfLoss,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    WHERE
        floodEvent NOT NULL
    GROUP BY
        floodEvent, yearOfLoss
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,floodEvent,yearOfLoss,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,Hurricane Katrina,2005,208348,16261697056,12659081935,3360020221,242594900
1,Hurricane Harvey,2017,92396,9055522699,6925370027,2115077279,15075393
2,Hurricane Sandy,2012,144848,8956609943,7707744797,951637071,297228075
3,Hurricane Ian,2022,48721,4757384580,4229314739,525543744,2526097
4,Hurricane Ike,2008,58126,2702511916,2073801567,577791589,50918760
5,Mid-summer severe storms,2016,30017,2533431383,2175218939,347263588,10948856
6,Hurricane Irene,2011,52493,1347399996,1139897981,183189121,24312894
7,Hurricane Ida,2021,28317,1346749832,1115951795,229650151,1147886
8,Hurricane Ivan,2004,20137,1325419294,1083795424,221959720,19664150
9,Hurricane Irma,2017,33339,1114668772,945821300,162148464,6699008


## Top 20 States and Flood Events by Total Paid Claims

In [39]:
con.sql("""
    SELECT
        state,
        floodEvent,
        yearOfLoss,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    WHERE
        floodEvent NOT NULL
    GROUP BY ALL
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,state,floodEvent,yearOfLoss,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,LA,Hurricane Katrina,2005,176276,13347164644,10441743931,2721451049,183969663
1,TX,Hurricane Harvey,2017,91870,9040181063,6912549831,2112555840,15075393
2,FL,Hurricane Ian,2022,47457,4711517234,4187737200,521388936,2391097
3,NJ,Hurricane Sandy,2012,74983,4372115685,3673280769,457640033,241194883
4,NY,Hurricane Sandy,2012,57405,4214352197,3702376632,465410811,46564754
5,LA,Mid-summer severe storms,2016,30017,2533431383,2175218939,347263588,10948856
6,MS,Hurricane Katrina,2005,19051,2521512578,1910623703,558996059,51892816
7,TX,Hurricane Ike,2008,44095,2230307106,1723323896,481446923,25536286
8,FL,Hurricane Irma,2017,28755,985551621,834411306,145153321,5986994
9,TX,Tropical Storm Allison,2001,26821,978096837,723017851,250198245,4880741


## Top 20 Rated Flood Zones by Total Paid Claims
From the Data Dictionary:
>Formerly called floodZone. NFIP Flood Zone derived from the Flood Insurance Rate Map (FIRM) used to rate the insured property. A - Special Flood with no Base Flood Elevation on FIRM; AE, A1-A30 - Special Flood with Base Flood Elevation on FIRM; A99 - Special Flood with Protection Zone; AH, AHB* - Special Flood with Shallow Ponding; AO, AOB* - Special Flood with Sheet Flow; X, B - Moderate Flood from primary water source. Pockets of areas subject to drainage problems; X, C - Minimal Flood from primary water source. Pockets of areas subject to drainage problems; D - Possible Flood; V - Velocity Flood with no Base Flood Elevation on FIRM; VE, V1-V30 - Velocity Flood with Base Flood Elevation on FIRM; AE, VE, X - New zone designations used on new maps starting January 1, 1986, in lieu of A1-A30, V1-V30, and B and C; AR - A Special Flood Hazard Area that results from the decertification of a previously accredited flood protection system that is determined to be in the process of being restored to provide base flood protection; AR Dual Zones - (AR/AE, AR/A1-A30, AR/AH, AR/AO, AR/A) Areas subject to flooding from failure of the flood protection system (Zone AR) which also overlap an existing Special Flood Hazard Area as a dual zone; *AHB, AOB, ARE, ARH, ARO, and ARA are not risk zones shown on a map, but are acceptable values for rating purposes*

In [40]:
con.sql("""
    SELECT
        ratedFloodZone as ratedFloodZone,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY ratedFloodZone
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,ratedFloodZone,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,AE,902694,35479231566,29574222634,5389301443,515707490
1,X,398598,13436983455,10383896784,3024472374,28614297
2,B,114822,3826110386,2835364624,971908875,18836887
3,A,198639,3130755422,2486940415,606541394,37273612
4,C,163832,2853210440,2119312629,723803760,10094051
5,A04,51806,2185871995,1800832671,364399597,20639726
6,A03,41872,1650992713,1337863894,284704083,28424737
7,VE,46418,1604044125,1427115498,158958851,17969776
8,A01,51362,1526019156,1188933368,320999694,16086094
9,A08,45657,1455607732,1185003260,237429765,33174708


## Total Paid Claims by Post-FIRM Construction Indicator
From the Data Dictionary:
>Indicates whether construction was started before or after publication of the FIRM. Yes is indicated with either a 'true' or '1'. No is indicated with either a 'false' or '0'. For insurance rating purposes, buildings for which the start of construction or substantial improvement was after December 31, 1974, or on or after the effective date of the initial FIRM for the community, whichever is later, are considered Post-FIRM construction.

In [41]:
con.sql("""
    SELECT
        postFIRMConstructionIndicator,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        postFIRMConstructionIndicator
    ORDER BY
        paidTotalClaim DESC
""").df()

,postFIRMConstructionIndicator,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,0,1961171,53017758311,43012194948,9378595126,626968236
1,1,710315,27620594292,22037398660,5261726761,321468871


## Total Paid Claims by Basement Enclosure Crawlspace Types
From the Data Dictionary:
>Basement is defined for purposes of the NFIP as any level or story which has its floor subgrade on all sides. Basement structure values are as follows: 0 - None; 1 - Finished Basement/Enclosure; 2 - Unfinished Basement/Enclosure; 3 - Crawlspace; 4 - Subgrade Crawlspace

In [42]:
con.sql("""
    SELECT
        basementEnclosureCrawlspaceType,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        basementEnclosureCrawlspaceType
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,basementEnclosureCrawlspaceType,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,NaN,1867992,55736598435,43974709709,11010313677,751575049
1,0.0,258080,14118916150,11621906529,2431677559,65332063
2,2.0,283047,6023112182,5309665486,621057933,92388762
3,1.0,218434,3411584798,2953210350,440219829,18154620
4,4.0,43933,1348141039,1190101534,137052891,20986614


## Total Paid Claims by Elevated Building Indicator
From the Data Dictionary:
>Indicates whether a building meets the NFIP definition of an elevated building. Yes is indicated with either a 'true' or '1'. No is indicated with either a 'false' or '0'. An elevated building is a no-basement building that was constructed so as to meet the following criteria: 1. The top of the elevated floor (all A zones) or the bottom of the lowest horizontal structural member of the lowest floor (all V zones) is above ground level. 2. The building is adequately anchored. 3. The method of elevation is pilings, columns (posts and piers), shear walls (not in V zones), or solid foundation perimeter walls (not in V zones).

In [43]:
con.sql("""
    SELECT
        elevatedBuildingIndicator,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        elevatedBuildingIndicator
    ORDER BY
        paidTotalClaim DESC
""").df()

,elevatedBuildingIndicator,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,0,2187042,66083874242,52806310817,12838681833,438881592
1,1,484444,14554478361,12243282792,1801640054,509555515


## Total Paid Claims by Occupancy Types
From the Data Dictionary:
>Code indicating the use and occupancy type of the insured structure. Note, 2-digit codes are for Risk Rating 2.0 policies. 1=single family residence; 2 = 2 to 4 unit residential building; 3 = residential building with more than 4 units; 4 = Non-residential building; 6 = Non Residential - Business; 11 = Single-family residential building with the exception of a mobile home or a single residential unit within a multi unit building; 12 = A residential non-condo building with 2, 3, or 4 units seeking insurance on all units; 13 = A residential non-condo building with 5 or more units seeking insurance on all units; 14 = Residential mobile/manufactured home; 15 = Residential condo association seeking coverage on a building with one or more units; 16 = Single residential unit within a multi-unit building; 17 = Non-residential mobile/manufactured home; 18 = A non-residential building; 19 = a non-residential unit within a multi-unit building

In [44]:
con.sql("""
    SELECT
        occupancyType,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        occupancyType
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,occupancyType,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,1.0,2085505,58074889678,46438196346,10745221948,891471384
1,4.0,174509,6399512277,4292656524,2096028112,10827641
2,3.0,93350,4398976557,4253333229,138336377,7306952
3,2.0,148432,3733231026,3410379315,287688446,35163265
4,6.0,44282,3360242402,2512042450,846579743,1620209
5,11.0,94093,2822556575,2447270648,373666665,1619262
6,15.0,8194,946313442,941970967,4252475,90000
7,18.0,8353,504682743,413963078,90689665,30000
8,12.0,4594,164787070,157682468,7040755,63847
9,13.0,1658,115997430,113511947,2485483,0


# Additional Analysis

## Total Paid Claims by Cause of Damage Code
From the Data Dictionary:
>Indicates the method by which the insured's property and contents were damaged. Legal values (value : description): 0 : Other causes; 1 : Tidal water overflow; 2 : Stream, river, or lake overflow; 3 : Alluvial fan overflow; 4 : Accumulation of rainfall or snowmelt; 7 : Erosion-demolition; 8 : Erosion-removal; 9 : Earth movement, landslide, land subsidence, sinkholes, etc.; A : Closed basin lake; B : Expedited claim handling process without site inspection; C : Expedited claim handling process follow-up site inspection; D : Expedited claim handling process by Adjusting Process Pilot Program (Remote Adjustment); NOTE: Due to certain provisions of the Upton Jones Amendment to the National Flood Insurance Act, cause of loss codes 7 and 8 may be used only if the date of loss is prior to September 23, 1995. More than one cause of loss code can be selected. For example, you may select 2 (stream, river, or lake overflow) and D (remote adjustment) on a single claim, or any combination of letters/numbers as appropriate

In [45]:
con.sql("""
    SELECT
        causeOfDamage,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        causeOfDamage
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,causeOfDamage,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,4,1165067,3.229711e+10,2.592098e+10,6.094557e+09,281581284.0
1,1,652400,2.115363e+10,1.734079e+10,3.445278e+09,367564742.0
2,2,535954,1.849811e+10,1.479851e+10,3.483557e+09,216043929.0
3,0,219652,6.152451e+09,4.980477e+09,1.103245e+09,68728023.0
4,B,15809,2.101842e+09,1.650532e+09,4.400003e+08,11309655.0
5,3,12895,2.366988e+08,1.849697e+08,4.949010e+07,2239014.0
6,None,60282,5.607188e+07,5.228969e+07,3.710770e+06,71424.0
7,C,430,4.599313e+07,3.679560e+07,9.158730e+06,38800.0
8,A,684,4.026256e+07,3.331573e+07,6.465500e+06,481325.0
9,7,602,2.275529e+07,2.233714e+07,3.205870e+05,97560.0


## Total Paid Claims by Flood Characteristics Indicator
From the Data Dictionary:
>Indicates characteristics of the flood waters. Legal values (value : description): 1 : Velocity Flow; 2 : Low-Velocity Flow or Ponding; 3 : Wave Action; 4 : Mudflow; 5 : Erosion;

In [46]:
con.sql("""
    SELECT
        floodCharacteristicsIndicator,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        floodCharacteristicsIndicator
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,floodCharacteristicsIndicator,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,NaN,2631633,80040375269,64608337763,14488023555,944013951
1,2.0,29123,392139926,284532994,105348196,2258735
2,1.0,8026,169487124,126607466,41429777,1449881
3,3.0,1818,28177518,23333657,4129321,714539
4,4.0,649,6794025,5504227,1289798,0
5,5.0,237,1378741,1277501,101240,0


## Total Paid Claims by Floodproofed Indicator
From the Data Dictionary:
>Indicates if the insured structure is floodproofed. Yes is indicated with either a 'true' or '1'. No is indicated with either a 'false' or '0'. NOTE: Floodproofing is any combination of structural and non-structural additions, changes, or adjustments to structures which reduce or eliminate flood damage to a property. Floodproofing may be an alternative to elevating a building to or above the BFE; however, the NFIP requires a Floodproofing Certificate prior to considering floodproofing mitigation measures in rating a structure

In [47]:
con.sql("""
    SELECT
        floodproofedIndicator,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        floodproofedIndicator
    ORDER BY
        paidTotalClaim DESC
""").df()

,floodproofedIndicator,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,0,2671109,80619514527,65035738829,14635368591,948407107
1,1,377,18838076,13854779,4953296,30000


## Total Paid Claims by Building Description Code
From the Data Dictionary:
>Indicates the description of the use of the insured building. Legal values (value : description): 01 : Main House; 02 : Detached Guest House; 03 : Detached Garage; 04 : Agricultural Building; 05 : Warehouse; 06 : Pool House, Clubhouse, Recreation Building; 07 : Tool/Storage Shed; 08 : Other; 09 : Barn; 10 : Apartment Building; 11 : Apartment - Unit; 12 : Cooperative Building; 13 : Cooperative - Unit; 14 : Commercial Building; 15 : Condominium (Entire Building); 16 : Condominium - Unit; 17 : House of Worship; 18 : Manufactured (Mobile) Home; 19 : Travel Trailer; 20 : Townhouse/Rowhouse;

In [48]:
con.sql("""
    SELECT
        buildingDescriptionCode,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        buildingDescriptionCode
    ORDER BY
        paidTotalClaim DESC
    LIMIT 20
""").df()

,buildingDescriptionCode,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,1.0,905145,39094112092,32540928483,6132748786,420434822
1,NaN,1717297,38230967250,29855329355,7851193461,524444435
2,8.0,37859,2754729151,2195108224,556837496,2783431
3,5.0,2367,199062000,130086791,68975209,0
4,10.0,2330,188592994,187206254,1386741,0
5,2.0,2258,58607574,49616808,8324777,665989
6,6.0,1324,56060632,47843930,8162650,54053
7,7.0,957,14025659,9901318,4095639,28703
8,3.0,760,13168321,11508338,1647109,12875
9,12.0,137,8563373,7334598,1215975,12800


## Total Paid Claims by Rental Property Indicator
From the Data Dictionary:
>Indicates if the property is a rental property. Yes is indicated with either a 'true' or '1'. No is indicated with either a 'false' or '0'. NOTE: A rental property is a property from which the owner receives payment from the occupant(s), known as tenants, in return for occupying or using the property. Rental properties may be either residential or commercial.

In [49]:
con.sql("""
    SELECT
        rentalPropertyIndicator,
        COUNT(id) AS countClaims,
        ROUND(
            SUM(amountPaidOnBuildingClaim)
            + SUM(amountPaidOnContentsClaim)
            + SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidTotalClaim,  
        ROUND(SUM(amountPaidOnBuildingClaim), 0)::BIGINT AS paidBuildingClaim,
        ROUND(SUM(amountPaidOnContentsClaim), 0)::BIGINT AS paidContentsClaim, 
        ROUND(SUM(amountPaidOnIncreasedCostOfComplianceClaim), 0)::BIGINT AS paidICC
    FROM
        claims
    GROUP BY
        rentalPropertyIndicator
    ORDER BY
        paidTotalClaim DESC
""").df()

,rentalPropertyIndicator,countClaims,paidTotalClaim,paidBuildingClaim,paidContentsClaim,paidICC
0,0,2638216,79198225176,63777570301,14474933322,945721553
1,1,33270,1440127427,1272023307,165388566,2715554
